In [37]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from pymongo import MongoClient
from tqdm import tqdm
import time
from datetime import datetime, timedelta
import re

In [38]:
# MongoDB 클라이언트 및 컬렉션 설정
client = MongoClient(host="", port=, username='', password='')
db = client['BOF2024_crawling']
collection = db['tiktok_crawling']

In [39]:
# 웹드라이버 초기화 및 네이버 카페 검색 페이지 열기
driver = webdriver.Chrome()
driver.get("https://www.tiktok.com/")
driver.maximize_window()

time.sleep(5) # 5초간 대기

In [40]:
# 검색어 입력 요소 찾기
search_input = driver.find_element(By.CSS_SELECTOR, 'input[name="q"]')

# 검색어 입력
search_input.send_keys("釜山亚洲艺术节")

# 엔터키 입력 
search_input.send_keys(Keys.RETURN)

# 10초간 대기 (2단계 인증)
time.sleep(10)

In [ ]:
# 숫자 값을 선택적으로 소수 및 K/M(천/백만) 접미사와 일치시키는 정규 표현식 패턴
pattern = r'(\d*\.?\d+)([KM])?'

# 조회수 요소 찾기
view_count_elements = driver.find_elements(By.CLASS_NAME, 'css-ws4x78-StrongVideoCount')

# 조회수 추출 및 저장
view_counts = []
for view_count_element in view_count_elements:
    try:
        view_count_text = view_count_element.text
        match = re.match(pattern, view_count_text)
        if match:
            number = float(match.group(1))  # 숫자 부분 추출
            suffix = match.group(2)
            if suffix == 'K':
                number *= 1000  # 'K' (천 단위)일 경우 x 1000 
            elif suffix == 'M':
                number *= 1000000  # 'M' (백만 단위)일 경우 x 1000000
            view_counts.append(int(number))
    except:
        pass

In [ ]:
# 오늘 날짜 설정
today = datetime.today()

def parse_date(date_text):
    try:
        if "일 전" in date_text:
            days = int(re.search(r'(\d+)일 전', date_text).group(1))
            date = today - timedelta(days=days)
        elif "주 전" in date_text:
            weeks = int(re.search(r'(\d+)주 전', date_text).group(1))
            date = today - timedelta(weeks=weeks)
        elif "시간 전" in date_text or "분 전" in date_text:
            date = today
        elif '-' in date_text:
            parts = date_text.split('-')
            if len(parts) == 2:  # mm-dd 형식
                month, day = int(parts[0]), int(parts[1])
                date = datetime(today.year, month, day)
            elif len(parts) == 3:  # yyyy-mm-dd 형식
                year, month, day = int(parts[0]), int(parts[1]), int(parts[2])
                date = datetime(year, month, day)
        else:
            date = today  # 기본적으로 오늘 날짜 할당

        return date.strftime('%Y.%m.%d.')
    except:
        return "N/A"

In [41]:
# 맨 아래까지 스크롤
SCROLL_PAUSE_TIME = 1.5
last_height = driver.execute_script("return document.documentElement.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.documentElement.scrollHeight;")
    if new_height == last_height:
        break
    last_height = new_height

In [42]:
title_divs = driver.find_elements(By.CLASS_NAME, 'css-1iy6zew-DivContainer')  # 제목 요소 찾기
url_divs = driver.find_elements(By.CLASS_NAME, 'css-at0k0c-DivWrapper')  # URL 요소 찾기
channel_name_divs = driver.find_elements(By.CLASS_NAME, 'css-dq7zy8-DivUserInfo')  # 채널 이름 요소 찾기
date_elements = driver.find_elements(By.CLASS_NAME, 'css-dennn6-DivTimeTag')  # 날짜 요소 찾기

total_data_count = len(title_divs)

# 제목, URL, 채널 이름, 조회수 및 날짜 추출 및 출력
data_list = []
for i, (title_div, url_div, channel_div) in enumerate(zip(title_divs, url_divs, channel_name_divs), start=1):
    try:
        # 모든 요소가 있는지 확인
        if not (title_div and url_div and channel_div and i <= len(view_counts) and i <= len(date_elements)):
            continue  # 요소 중 하나라도 없으면 다음 항목으로 건너뛰기

        # 제목 추출
        title_texts = []
        for element in title_div.find_elements(By.XPATH, './/*'):
            if element.tag_name == 'a':
                aria_label = element.get_attribute('aria-label')
                if aria_label and '#' in aria_label:
                    try:
                        title_texts.append('#' + aria_label.split('#')[1].split()[0])
                    except IndexError:
                        continue  
            else:
                title_texts.append(element.text)
        title = ' '.join(title_texts)

        # URL 추출
        url_texts = []
        for element in url_div.find_elements(By.TAG_NAME, 'a'):
            url_texts.append(element.get_attribute('href'))
        url = ' '.join(url_texts)

        # 채널 이름 추출
        channel_names = []
        for channel_div_element in channel_div.find_elements(By.CLASS_NAME, 'css-2zn17v-PUniqueId'):
            channel_names.append(channel_div_element.text)

        # 날짜 추출 및 변환
        raw_date_text = date_elements[i-1].text
        date_text = parse_date(raw_date_text)

        # 데이터 리스트에 추가
        for channel_name in channel_names:
            data_list.append({
                "제목": title,
                "URL": url,
                "채널명": channel_name,
                "조회수": view_counts[i-1],
                "업로드 날짜": date_text,
                "검색키워드": "釜山亚洲艺术节",
                "설명": title
            })

    except:
        continue

# MongoDB에 데이터 적재
for data in tqdm(data_list, desc="데이터 적재 진행 중", unit="데이터"):
    collection.insert_one(data)

데이터 적재 진행 중: 100%|██████████| 134/134 [00:01<00:00, 98.16데이터/s] 
